In [178]:
from google.colab import files
uploaded = files.upload()

Saving Data.csv to Data (1).csv


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 61.5 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [179]:
import pandas as pd
df= pd.read_csv('Data.csv')


In [180]:
df.ADR_label.nunique()

2

In [181]:
df.shape

(23516, 4)

In [183]:
df.head()

,UserId,TweetId,Tweet,ADR_label
0,10030778,413205,Intravenous azithromycin-induced ototoxicity.,1
1,10048291,528244,"Immobilization, while Paget's bone disease was...",1
2,10048291,361834,Unaccountable severe hypercalcemia in a patien...,1
3,10082597,292240,METHODS: We report two cases of pseudoporphyri...,1
4,10082597,467101,METHODS: We report two cases of pseudoporphyri...,1


In [184]:
df['num_words'] = df.Tweet.apply(lambda x:len(x.split()))

In [185]:
df['num_words'].max()

135

In [186]:
df['ADR_label']= df['ADR_label'].astype('category')

In [187]:
from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(df, test_size = 0.3, random_state = 42)

In [188]:
from tensorflow.keras.utils import to_categorical

In [189]:
to_categorical(data_train.ADR_label)

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

##BERT Model

In [14]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [32]:
tokenizer.save_pretrained('bert-tokenizer')
bert.save_pretrained('bert-model')

In [29]:
tokenizer('hello this me abhishek')

{'input_ids': [101, 19082, 1142, 1143, 170, 1830, 27516, 4638, 1377, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [190]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=data_train.Tweet.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=data_test.Tweet.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [74]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

In [193]:
max_len = 100
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)

out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(256, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(128,activation = 'relu')(out)

y = Dense(2,activation = 'softmax')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)

In [194]:
model.layers[2].trainable = True

In [195]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = ['accuracy'])

In [196]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    multiple             108310272   ['input_ids[0][0]',              
                                                                  'attention_mask[0][0]']         
                                                                                                  
 global_max_pooling1d_10 (Globa  (None, 768)         0           ['tf_bert_model[10][0]']   

In [152]:
model.layers

In [198]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train.ADR_label),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.ADR_label)
    ),
  epochs=3,
    batch_size=32
)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


515/515 [==============================] - 406s 741ms/step - loss: 0.1403 - accuracy: 0.9478 - val_loss: 0.0875 - val_accuracy: 0.9680
Epoch 2/3
515/515 [==============================] - 380s 738ms/step - loss: 0.0568 - accuracy: 0.9810 - val_loss: 0.0946 - val_accuracy: 0.9677
Epoch 3/3
515/515 [==============================] - 380s 738ms/step - loss: 0.0292 - accuracy: 0.9906 - val_loss: 0.1120 - val_accuracy: 0.9684


In [31]:
model.save_weights('ADR_weights.h5')

In [199]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

In [200]:
import numpy as np
y_predicted = np.argmax(predicted_raw, axis = 1)

In [201]:
predicted_raw[0]

array([9.9989772e-01, 1.0228796e-04], dtype=float32)

In [202]:
y_predicted[0]

0

In [38]:
from sklearn.metrics import classification_report


In [203]:
print(classification_report(data_test.ADR_label, y_predicted))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      5023
           1       0.93      0.96      0.95      2032

    accuracy                           0.97      7055
   macro avg       0.96      0.97      0.96      7055
weighted avg       0.97      0.97      0.97      7055



In [40]:
texts = input(str('input the text'))

x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
validation

input the textClevudine (L-FMAU) is a\ thymidine l-nucleoside analogue that was recently introduced for the treatment of chronic hepatitis B virus infection.


array([[98.3765   ,  1.6234894]], dtype=float32)

In [42]:
encoded_dict  = {'ADR_No':0,'ADR_Yes':1}

In [43]:
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

ADR_No 98.3765
ADR_Yes 1.6234894


## DistillBert

In [208]:
from transformers import DistilBertTokenizer, TFDistilBertModel
DistilBert_Tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [205]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = DistilBert_Tokenizer(
    text=data_train.Tweet.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = DistilBert_Tokenizer(
    text=data_test.Tweet.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [209]:
max_len = 100
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = dbert_model(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)

out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(256, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(128,activation = 'relu')(out)

y = Dense(2,activation = 'softmax')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)

In [210]:
model.layers[2].trainable = True

In [211]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = ['accuracy'])

In [212]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 100, 768),                                                   
                                 hidden_states=None                                         

In [213]:
model.layers

In [214]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train.ADR_label),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.ADR_label)
    ),
  epochs=3,
    batch_size=32
)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


515/515 [==============================] - 205s 377ms/step - loss: 0.2438 - accuracy: 0.8961 - val_loss: 0.1555 - val_accuracy: 0.9409
Epoch 2/3
515/515 [==============================] - 193s 375ms/step - loss: 0.1227 - accuracy: 0.9541 - val_loss: 0.1428 - val_accuracy: 0.9480
Epoch 3/3
515/515 [==============================] - 194s 377ms/step - loss: 0.0844 - accuracy: 0.9702 - val_loss: 0.1428 - val_accuracy: 0.9505


##Roberta Model


In [219]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

roberta_tokenizer = AutoTokenizer.from_pretrained(MODEL)
roberta_model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [220]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = roberta_tokenizer(
    text=data_train.Tweet.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = roberta_tokenizer(
    text=data_test.Tweet.tolist(),
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [221]:
max_len = 100
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = dbert_model(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)

out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(256, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(128,activation = 'relu')(out)

y = Dense(2,activation = 'softmax')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)

In [222]:
model.layers[2].trainable = True

In [223]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = ['accuracy'])

In [224]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 100)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 100, 768),                                                   
                                 hidden_states=None                                         

In [225]:
model.layers

In [226]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train.ADR_label),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test.ADR_label)
    ),
  epochs=3,
    batch_size=32
)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


515/515 [==============================] - 206s 379ms/step - loss: 0.4907 - accuracy: 0.7643 - val_loss: 0.3734 - val_accuracy: 0.8344
Epoch 2/3
515/515 [==============================] - 193s 376ms/step - loss: 0.3510 - accuracy: 0.8460 - val_loss: 0.3383 - val_accuracy: 0.8500
Epoch 3/3
515/515 [==============================] - 193s 376ms/step - loss: 0.3029 - accuracy: 0.8701 - val_loss: 0.3249 - val_accuracy: 0.8578
